In [2]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process

In [3]:
# Copias de los DataFrames
df_productos = pd.read_csv("L_STMaestroPrd.csv")
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_9920\2324721240.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_productos = pd.read_csv("L_STMaestroPrd.csv")


In [4]:
"""
Este modulo estandariza los csv que provienen del proveedor.
"""
# Función para procesar el caso de "varios productos en una fila"
def procesar_varios_productos(df):
    # Iteramos por todas las columnas del DataFrame
    for column in df.columns:
        # Convertimos a listas si hay '\n'
        df[column] = df[column].apply(
            lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan])
        )

    # Crear una lista vacía para almacenar las filas explotadas
    df_exploded_list = []

    # Explosión fila por fila
    for _, row in df.iterrows():
        row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]
        max_len = max(len(lst) for lst in row_lists)
        row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]
        exploded_row = pd.DataFrame(row_lists).T
        exploded_row.columns = df.columns
        df_exploded_list.append(exploded_row)

    # Concatenar todas las filas explotadas
    return pd.concat(df_exploded_list, ignore_index=True)

# Función para procesar el caso de "un producto por fila"
def procesar_un_producto(df):
    # Limpiar saltos de línea en todas las columnas
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x
        )
    return df

# Función principal para decidir el módulo a usar
def procesar_csv(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path)

    # Comprobar si la primera columna contiene '\n'
    contiene_saltos = df.iloc[:, 0].apply(lambda x: '\n' in x if isinstance(x, str) else False).any()

    if contiene_saltos:
        print("Usando el módulo: VARIOS PRODUCTOS EN UNA FILA")
        return procesar_varios_productos(df)
    else:
        print("Usando el módulo: UN PRODUCTO POR FILA")
        return procesar_un_producto(df)


# RUTA CSV DEL PROVEEDOR
ruta_csv = "prueba.csv"
df_resultado_PROVEEDOR = procesar_csv( ruta_csv)

# df estandarizado
df_resultado_PROVEEDOR
# Limpiar las descripciones en ambas tablas del Código 
df_resultado_PROVEEDOR['descripcion_limpia'] = df_resultado_PROVEEDOR['DESCRIPCIÓN'].fillna('').apply(lambda x: x.lower())


Usando el módulo: UN PRODUCTO POR FILA


In [5]:
# Función de limpieza de texto para una columna específica del DataFrame
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.
    """
    # Eliminamos palabras irrelevantes como unidades, envases, etc.
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    # Reemplazar saltos de línea por espacios
    columna_limpia = re.sub(r'\n', ' ', columna_limpia)
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    columna_limpia = columna_limpia.lower()
    # Eliminamos palabras comunes (como 'de', 'la', etc.) que no ayudan a la comparación
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|ac|se)\b', '', columna_limpia).strip()
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)
    # Aseguramos que unidades como mg, ml, etc. no interfieran
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)
    return columna_limpia

In [6]:
# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()  # Usar .copy() para evitar la advertencia

    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        # Ahora podemos modificar la copia sin generar la advertencia
        tablas_coincidencias['descrip1_limpia'] = tablas_coincidencias['descrip1'].fillna('').apply(lambda x: x.lower())
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

column_name_laboratorio = "atrib0"
laboratorio_coincidencias_df = laboratorio_input_df(df_productos, column_name_laboratorio)


Se encontraron 100 productos coincidentes del laboratorio 'abbv'.


In [7]:
if laboratorio_coincidencias_df is not None:
    proveedor_index = df_resultado_PROVEEDOR['descripcion_limpia'].tolist()

    matches = []

    for _, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        all_matches = process.extract(prov_desc, laboratorio_coincidencias_df['descrip1_limpia'], scorer=fuzz.partial_ratio, limit=None)
        valid_matches = [match for match in all_matches if match[1] >= 80]

        if valid_matches:
            for match in valid_matches:
                matched_row = laboratorio_coincidencias_df[
                    laboratorio_coincidencias_df['descrip1_limpia'] == match[0]
                ].iloc[0]
                match_info = prov_row.to_dict()
                match_info.update(matched_row.to_dict())
                match_info['similaridad'] = match[1]
                matches.append(match_info)
        else:
            match_info = prov_row.to_dict()
            match_info['similaridad'] = 0
            for col in laboratorio_coincidencias_df.columns:
                if col not in match_info:
                    match_info[col] = None
            matches.append(match_info)

    df_matches = pd.DataFrame(matches)

    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")



Coincidencias encontradas: 0
    similaridad            EAN  \
0             0  7791939000089   
1             0  7791939001116   
2             0  7791939000096   
3             0  7791939000133   
4             0  7791939001383   
5             0  7791939001390   
6             0  7791939001413   
7             0  7791939001437   
8             0  7791939000416   
9             0  7791939000522   
10            0  7791939000928   
11            0  7791939000218   
12            0  7791939000140   
13            0  7791939000171   
14            0  7791939000751   
15            0  7791939000911   
16            0  7791939000614   

                                          DESCRIPCIÓN          PVP  \
0   ASPIRINA 100 Estuche: 10 blisters x 10 comp. x...    $4.904,38   
1           CARDIO VENT3 30 comp masticables x 100 mg        $1,31   
2   CALCIO BASE Estuche: 3 blisters x 20 comp. mas...   $12.995,10   
3   CARBON VEGETAL ACTIVADO Estuche: 10 blisters x...   $14.071,69   
4   CLO

In [8]:
df_matches

,similaridad,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO,descripcion_limpia,niprod,cod_prod,...,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia
0,0,7791939000089,ASPIRINA 100 Estuche: 10 blisters x 10 comp. x...,"$4.904,38","$2.305,06",53%,50,aspirina 100 estuche: 10 blisters x 10 comp. x...,None,None,...,None,None,None,None,None,None,None,None,None,None
1,0,7791939001116,CARDIO VENT3 30 comp masticables x 100 mg,"$1,31",$393,70%,50,cardio vent3 30 comp masticables x 100 mg,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0,7791939000096,CALCIO BASE Estuche: 3 blisters x 20 comp. mas...,"$12.995,10","$6.107,70",53%,90,calcio base estuche: 3 blisters x 20 comp. mas...,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0,7791939000133,CARBON VEGETAL ACTIVADO Estuche: 10 blisters x...,"$14.071,69","$6.613,69",53%,50,carbon vegetal activado estuche: 10 blisters x...,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0,7791939001383,CLONAZEPAM SL ...,"$ 6.208,60","$ 1.738,41",72%,40,clonazepam sl ...,None,None,...,None,None,None,None,None,None,None,None,None,None
5,0,7791939001390,CLONAZEPAM ...,"$ 11.824,64","$ 3.310,90",72%,40,clonazepam ...,None,None,...,None,None,None,None,None,None,None,None,None,None
6,0,7791939001413,CLONAZEPAM ...,"$ 11.326,00","$ 3.171,28",72%,40,clonazepam ...,None,None,...,None,None,None,None,None,None,None,None,None,None
7,0,7791939001437,CLONAZEPAM ...,"$ 12.593,93","$ 3.526,30",72%,40,clonazepam ...,None,None,...,None,None,None,None,None,None,None,None,None,None
8,0,7791939000416,IBUPROFENO 400 Estuche: 2 blisters x 10 comp. ...,"$3.100,34","$868,10",72%,50,ibuprofeno 400 estuche: 2 blisters x 10 comp. ...,None,None,...,None,None,None,None,None,None,None,None,None,None
9,0,7791939000522,IBUPROFENO 600 Estuche: 2 blisters x 10 comp. ...,"$6.567,98","$1.839,03",72%,40,ibuprofeno 600 estuche: 2 blisters x 10 comp. ...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
df_matches["DESCRIPCIÓN"]

0     ASPIRINA 100 Estuche: 10 blisters x 10 comp. x...
1             CARDIO VENT3 30 comp masticables x 100 mg
2     CALCIO BASE Estuche: 3 blisters x 20 comp. mas...
3     CARBON VEGETAL ACTIVADO Estuche: 10 blisters x...
4     CLONAZEPAM SL                                 ...
5     CLONAZEPAM                                    ...
6     CLONAZEPAM                                    ...
7     CLONAZEPAM                                    ...
8     IBUPROFENO 400 Estuche: 2 blisters x 10 comp. ...
9     IBUPROFENO 600 Estuche: 2 blisters x 10 comp. ...
10       IBUPROFENO Susp. 4% Estuche: 1 frasco x 90 ml.
11    PARACETAMOL 500 Estuche: 2 blisters x 10 comp....
12         PARACETAMOL GOTAS Estuche: 1 gotero x 20 ml.
13    SALES REHIDRATACION ORAL                  Estu...
14    SALES REHIDRATACION ORAL FRUTAL Estuche: 3 sob...
15                    OMEPRAZOL VL 14 cápsulas x 20 mg.
16                 ENALAPRIL Estuche: 30 comp. x 10 mg.
Name: DESCRIPCIÓN, dtype: object